# 2013-2017 American Community Survey (ACS) five-year estimates 

By [Christine Zhang](mailto:czhang@baltsun.com)

## Poverty

### Import R data analysis libraries

In [2]:
suppressMessages(library('tidyverse'))
suppressMessages(library('tidycensus'))

### "According to the ACS, 14.6 percent of Americans lived below the poverty threshold during the 2013-17 period. That’s lower than the 2008-12 poverty rate of 14.9 percent."

These stats are from the Census Bureau [press release](https://www.census.gov/newsroom/press-releases/2018/2013-2017-acs-5year.html) under "Poverty."

### "Maryland’s overall poverty rate rose to 9.7 percent from 9.4 percent."

We will use the `tidycensus` package by [Kyle Walker](https://twitter.com/kyle_e_walker) to get poverty rates for Maryland (you'll need an API Key from http://api.census.gov/data/key_signup.html). For more about `tidycensus`, click [here](https://walkerke.github.io/tidycensus/index.html). For more about Census APIs, click [here](https://www.census.gov/data/developers/data-sets.html).

In [74]:
# v17 <- load_variables(2017, 'acs5/subject', cache = TRUE) 
# v17 %>% filter(grepl('Percent below poverty level', label))

pov_perc_17 = get_acs(geography = 'state', 
                      year = 2017,
                      variables = 'S1701_C03_001',
                      state = "MD")

pov_perc_12 = get_acs(geography = 'state', 
                      year = 2012,
                      variables = 'DP03_0128P',
                      state = "MD")

Getting data from the 2013-2017 5-year ACS
Using the ACS Subject Tables
Getting data from the 2008-2012 5-year ACS
Using the ACS Data Profile


Merge `pov_perc_17` with `pov_perc_12` to create `pov_perc.merged`. 

In [75]:
pov_perc.merged <- merge(pov_perc_17, pov_perc_12, by = c('NAME'), 
                   suffixes = c('_17', '_12'))

In [76]:
print(paste("Maryland's poverty rate according to the 2013-17 ACS was",
            pov_perc.merged$estimate_17,
            "percent. In the 2008-12 ACS, it was",
            pov_perc.merged$estimate_12, "percent."))

[1] "Maryland's poverty rate according to the 2013-17 ACS was 9.7 percent. In the 2008-12 ACS, it was 9.4 percent."


Check if the difference is significant at the 90 percent confidence level. See ["Understanding and Using American Community Survey Data: What All Data Users Need to Know"](https://www.census.gov/programs-surveys/acs/guidance/handbooks/general.html) (starting from page 44) for more information.

In [77]:
pov_perc.merged <- pov_perc.merged %>% mutate(se_12 = moe_12/1.645,
                                              se_17 = moe_17/1.645,
                                              diff = estimate_17 - estimate_12,
                                              se_diff = sqrt(se_12^2 + se_17^2),
                                              t_diff = diff/se_diff,
                                              signif = ifelse(abs(t_diff) > 1.645, 1, 0))

pov_perc.merged

NAME,GEOID_17,variable_17,estimate_17,moe_17,GEOID_12,variable_12,estimate_12,moe_12,se_12,se_17,diff,se_diff,t_diff,signif
Maryland,24,S1701_C03_001,9.7,0.1,24,DP03_0128P,9.4,0.2,0.1215805,0.06079027,0.3,0.1359312,2.206999,1


The absolute value of `t_diff` is greater than 1.645 indicating significance at the 90 percent confidence level.

### "The overall poverty rate is still twice as high for black residents as it is for white residents, when comparing the five-year ACS periods."

We will use the `tidycensus` package to get poverty rates for Maryland for black and non-Hispanic/Latino white residents in the 2013-17 ACS. For more information about race/ethnicity categories in the Census, click [here](http://www.pewsocialtrends.org/2015/06/11/chapter-1-race-and-multiracial-americans-in-the-u-s-census/).

In [141]:
pov_perc_race_17 = get_acs(geography = 'state', 
                           year = 2017,
                           variables = c(black = 'S1701_C03_014',
                                         white_nhl = 'S1701_C03_021'),
                           state = "MD") 

Getting data from the 2013-2017 5-year ACS
Using the ACS Subject Tables


In [139]:
pov_perc_race_17

GEOID,NAME,variable,estimate,moe
24,Maryland,black,14.1,0.3
24,Maryland,white_nhl,6.6,0.2


In [138]:
print(paste('The overall poverty rate for black residents in Maryland is', 
            round(pov_perc_race_17[pov_perc_race_17$variable == 'black', ]$estimate/
                  pov_perc_race_17[pov_perc_race_17$variable == 'white_nhl', ]$estimate, 1),
            'times as high as it is for white residents, according to the 2103-17 ACS.'))

[1] "The overall poverty rate for black residents in Maryland is 2.1 times as high as it is for white residents, according to the 2103-17 ACS."


2008-12 ACS poverty rates for black and non-Hispanic/Latino white residents in Maryland were 14.5 percent and 6.2, respectively. These numbers can be found on the Census Bureau's American Factfinder tool [here](https://factfinder.census.gov/bkmk/table/1.0/en/ACS/12_5YR/S1701/0400000US24).